# 基于莫兰指数计算submask

In [ ]:
from pixelmed_calc.medical_imaging.RadiologyComponents.extrafeature import extract_image_and_mask_paths_from_directory
from pixelmed_calc.medical_imaging.RadiologyComponents.HabitatMorans import compute_Morans
import os
#配置参数

maskout=r"D:\pixelmedAI\function_pm\pm_data\liangxing\molan_submask1"

os.makedirs(maskout,exist_ok=True)
path_str=r"D:\pixelmedAI\function_pm\pm_data\liangxing"
method_str='directory'
images_str='images'
masks='masks'

#运行代码-----
images, masks = extract_image_and_mask_paths_from_directory(
            path=path_str,
            method=method_str,  # 默认为 'directory'
            images=images_str,  # 默认为 'images'
            masks=masks,    # 默认为 'masks'
        )

for img ,mask in zip(images,masks):
    print("正在计算",img)
    try:
        compute_Morans(img, mask,  os.path.join(maskout,os.path.basename(mask)),)
   
    except:
        print(f'请检查 图像以及mask {img}')

# 计算全局莫兰指数

In [ ]:
from pixelmed_calc.medical_imaging.RadiologyComponents.extrafeature import extract_image_and_mask_paths_from_directory
from pixelmed_calc.medical_imaging.RadiologyComponents.HabitatMorans import global_moran_to_csv
import os
#配置参数


path_str=r"D:\pixelmedAI\function_pm\pm_data\liangxing"
method_str='directory'
images_str='images'
masks='molan_submask'

#运行代码-----
images, masks = extract_image_and_mask_paths_from_directory(
            path=path_str,
            method=method_str,  # 默认为 'directory'
            images=images_str,  # 默认为 'images'
            masks=masks,    # 默认为 'masks'
        )
     
global_moran_to_csv(images,masks,'global_molan.csv')

在增强CT影像数据中，这四个亚区域可以通过影像强度值以及局部和周围区域的空间相关性，来反映一些特定的病理学或生理学意义。以下是基于增强CT影像的具体解释：

---

### **1. High-High 区域**（高强度聚集区，Hot Spot - 高值正相关）

#### **定义：**
- **局部莫兰指数为正**（`I > 0`）：这意味着当前像素与其邻域像素之间存在 **正相关性**。
- 当前像素值 **高于均值**（`local_value > mean_x`）。
- 它表示像素强度高，并且周围邻域像素值也同样较高。

#### **结合增强CT的解释：**
- **可能代表：**
  - 病理性高血流区域，例如：
    - **肿瘤富血区**（如肝癌、肾细胞癌等高血供肿瘤的强化区域）。
    - **炎症区** 或 **局部炎症反应**（如脓肿壁、增强的感染性病灶）。
  - **正常高血流区域**：
    - 器官强化正常区域（如肝动脉期时增强的肝组织或肾皮质）。
  - **造影剂浓集热点**：如造影剂泄漏（造影剂外渗）或与血管密集区域相关的现象。

---

### **2. Low-Low 区域**（低强度聚集区，Cold Spot - 低值正相关）

#### **定义：**
- **局部莫兰指数为正**（`I > 0`）：这意味着当前像素与邻域像素具有 **正相关性**。
- 当前像素值 **低于均值**（`local_value < mean_x`）。
- 它表示像素强度低，并且周围邻域像素值也同样较低。

#### **结合增强CT的解释：**
- **可能代表：**
  - **缺血性区域**：
    - 如梗死、缺血灶，这些区域的血供减少，导致对比剂强化低。
  - **低血流区域**：
    - 如肿瘤内部坏死区域（中央低密度区域），或某些肝转移瘤周围强化而中央强化缺失。
  - **液体/无血流区域**：
    - 如囊肿、脓肿、腔隙性积液区域。

---

### **3. High-Low 区域**（高-低不一致区域，Hot Spot Anomaly - 负相关）

#### **定义：**
- **局部莫兰指数为负**（`I < 0`）：当前像素与周围邻域之间存在 **负相关性**。
- 当前像素值 **高于区域均值**（`local_value > mean_x`）。
- 它表示当前像素值较高，但周围邻域像素值较低。

#### **结合增强CT的解释：**
- **可能代表：**
  - **异常高密度中心**：
    - 如局部的 **造影剂外渗现象**（显著对比剂积聚的点状高密度区域）。
  - **强化边界区域**：
    - 如坏死病灶周围的强化边，或者病灶边缘复制高血流灶（如部分肿瘤边缘强化，坏死中空区域的边缘区域高密度）。
  - **血管相关强化**：
    - 单一血管向周围发散，中心增强强而周边稀疏。

---

### **4. Low-High 区域**（低-高不一致区域，Cold Spot Anomaly - 负相关）

#### **定义：**
- **局部莫兰指数为负**（`I < 0`）：当前像素与周围邻域之间存在 **负相关性**。
- 当前像素值 **低于区域均值**（`local_value < mean_x`）。
- 它表示当前像素值较低，但周围邻域像素值较高。

#### **结合增强CT的解释：**
- **可能代表：**
  - **异常低密度中心**：
    - 如 **坏死中心区域**：肿瘤内部强化低的坏死区，与周围高密度的肿瘤强化边缘形成反差。
    - **脓肿中心：**脓肿壁强化但中心低密度（液体积聚）。
  - **低血流中心**：
    - 如较大的动脉栓塞性病灶，其中外围会显示高强化，而中心呈低密度。
  - **强化边界周边异常**：
    - 如增强良性的表现，但局部外围组织因病理变化（如纤维化）强化下降。

---

### **结合背景分析的补充：**

从增强CT中这四个亚区域的划分，提供了一种思路，用于区分和分析不同类型的组织反应、病变特征以及动态的血流变化：

1. **高强化区域**（如血供丰富的组织或异常强化点），可以快速定位可能的病理性区域（如肿瘤富血供）。
2. **低强化区域**（如缺血、坏死、液性病灶）提示低血流、病灶退行性病变的特征。
3. **负相关区域**（如“高-低”或“低-高”）可进一步提示局部密度与周围背景的差异，往往是异常区域的特征，如坏死、瘤周边或炎症反应。
4. **背景设置**的作用：通过确定不同阈值（如莫兰指数的显著性），增强CT分析可以从单纯的影像观察转变为更精确的影像数据统计推断（提供更有力的诊断依据）。

---

### **总结**：
在增强CT分析中，这四个亚区域可以帮助临床医生：
- 定位特定的病变（如肿瘤和炎性区域）。
- 辅助评估病变的性质（如高血供、低血供，或者坏死等）。
- 提供精确的空间相关性，支持更有效的诊断和治疗决策。


# 莫兰生境亚区域方法
为了生成感兴趣区域的亚区域，基于局部空间自相关性和区域属性值的方法被采用，以识别区域内具有显著差异的空间模式。具体过程如下：

首先，从输入的3D医学图像数据和对应的掩膜文件中加载数据，并提取各个感兴趣区域的定义。输入图像所提供的是空间属性值，例如信号强度或其他定量测量值，掩膜文件则标记不同区域的标签。这些输入数据用于分析不同掩膜区域的局部空间特征。

对于每一个感兴趣区域，通过掩膜提取其对应的体素(像素)集合，并记录这些体素的空间坐标和属性值。假设区域的体素集合为 $\{x_i | i = 1, ..., n\}$，其中 $x_i$ 是每个体素的属性值，且 $n$ 表示该区域内总的体素数。随后，计算区域内体素属性值的均值$\bar{x}$，作为基本的全局统计描述。区域内总体方差 $$M_2 = \frac{1}{n}\sum_{i=1}^{n}(x_i - \bar{x})^2$$  也同时计算，以作为标准化因子。

在空间分析中，为捕捉区域内体素之间的局部空间关联性，构建了一个基于邻域关系的权重矩阵 $W$。该权重矩阵的计算基于体素之间的空间距离，利用欧几里得距离判断体素是否为邻域关系。例如，如果两体素的距离小于等于其八邻域连接的对角距离 $\sqrt{3}$，则定义它们为邻居，权重 $w_{ij} = 1$，否则 $w_{ij} = 0$。通过这些定义，权重矩阵 $W$ 为一个对称的稀疏矩阵，表示区域内局部体素的空间关系。

在区域内的每个体素 $i$ 上，计算其局部莫兰指数。局部莫兰指数的表达式为：

$$ 
I_i = \frac{(x_i - \bar{x})}{M_2} \sum_{j} w_{ij}(x_j - \bar{x})
$$ 

其中 $x_i$ 和 $x_j$ 分别是体素 $i$ 和其邻域体素的属性值，$\bar{x}$ 是区域内属性值的均值，$M_2$ 是区域方差，$w_{ij}$ 来自权重矩阵 $W$。局部莫兰指数 $I_i$ 描述了体素 $i$ 和其邻域之间的空间关联性。正值的 $I_i$ 代表体素属性值与邻域的值呈现相似分布，属于空间正相关；负值的 $I_i$ 则代表体素与其邻域表现出属性值的显著差异，属于空间负相关。

基于局部莫兰指数和体素的属性值，为每个体素重新分类并生成新的掩膜。判别的逻辑如下：首先，判断 $I_i$ 的绝对值是否低于设定的显著性阈值，若是，则定义该体素所在位置为非显著区；否则，根据 $I_i$ 的符号进行进一步分类。如果 $I_i > 0$，即表现空间正相关，并结合体素值 $x_i$ 与均值 $\bar{x}$ 的关系，将其区分为高值-高值区 (High-High) 或低值-低值区 (Low-Low)。如果 $I_i < 0$，即表现空间负相关，则根据 $x_i$ 与 $\bar{x}$ 的关系，将其划分为高值-低值区 (High-Low) 或低值-高值区 (Low-High)。最终，生成新的掩膜文件，其中每个体素根据其局部属性值和空间相关性被重新划分为五类：高值-高值、低值-低值、高值-低值、低值-高值和非显著区。

完成每个感兴趣区域的分类后，将所有区域合并为全局掩膜，同时保留空间坐标信息和分类标签。最后，生成的掩膜被保存为标准的NIfTI格式，以便于进一步的分析和可视化应用。此方法通过结合局部空间自相关性分析和属性值分类，有效地在较大区域内识别空间异质性，从而提取亚区域特征。"
